In [1]:
import pandas as pd
import numpy as np
import seaborn as sns



In [2]:
df = pd.read_csv('C:\\Users\\Faisal\\Documents\\Data Science project\\results_1.csv')
df.tail()


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
41535,2019-11-19,Kyrgyzstan,Tajikistan,1,1,FIFA World Cup qualification,Bishkek,Kyrgyzstan,False
41536,2019-11-19,Vietnam,Thailand,0,0,FIFA World Cup qualification,Hanoi,Vietnam,False
41537,2019-11-19,Malaysia,Indonesia,2,0,FIFA World Cup qualification,Kuala Lumpur,Malaysia,False
41538,2019-11-19,Turkmenistan,Sri Lanka,2,0,FIFA World Cup qualification,Ashgabat,Turkmenistan,False
41539,2019-11-19,Lebanon,North Korea,0,0,FIFA World Cup qualification,Beirut,Lebanon,False


In [3]:
#making results columns
df['home_win'] = np.where(df['home_score'] > df['away_score'], 1, 0)
df['away_win'] = np.where(df['away_score'] > df['home_score'], 1, 0)
df['draw'] = np.where(df['away_score'] == df['home_score'], 1, 0)

#Assigning a match Id for every match
matches_num = list(df.shape)
matches_num = matches_num[0]
match_Id = list(range(1,matches_num+1))
df.insert(0,'match_Id',match_Id)
df.head()

#getting a list of all teams and assigning each a unique team Id
All_Teams = pd.unique(df[['home_team', 'away_team']].values.ravel('K'))
All_Teams.sort()
Teams_df = pd.DataFrame({'Team':All_Teams})
Id = list(range(1,len(All_Teams)+1))
Teams_df.insert(0,'Id',Id)

#Left joining Team Id's onto home and away teams for every match
Merge_A = pd.merge(df,
               Teams_df[['Id','Team']],
               left_on = 'home_team',
               right_on = 'Team',
               how = 'left')
Merge_A.rename(columns={'Id':'home_team_Id'}, inplace = True)
Merge_A.drop('Team', 1, inplace=True)

Merge_B = pd.merge(Merge_A,
               Teams_df[['Id','Team']],
               left_on = 'away_team',
               right_on = 'Team',
               how = 'left')
Merge_B.rename(columns={'Id':'away_team_Id'}, inplace = True)
Merge_B.drop('Team', 1, inplace=True)
Merge_B.head()


df_2 = Merge_B[['match_Id','date','home_team_Id','home_team','away_team_Id','away_team','home_score','away_score',
              'country','city','tournament','neutral','home_win','away_win','draw']]



In [4]:
#Making points from Last 3 & 6 games 'form' columns for every team, at every date


#Getting match points per game for every team

home_df2 = df_2[['match_Id','date','home_team_Id','home_team','home_win','draw']]
home_df2['Win_Points'] = list(np.where(df.loc[:,'home_win'] > 0, 3, 0))
home_df2['Draw_Points'] = np.where(df.loc[:,'draw'] > 0, 1, 0)
home_df2['Match_Points'] = home_df2['Win_Points'] + home_df2['Draw_Points']
home_df3 = home_df2[['match_Id','date','home_team_Id','Match_Points']]
home_df3.rename(columns={'home_team_Id':'team_Id'}, inplace = True)


away_df2 = df_2[['match_Id','date','away_team_Id','away_team','away_win','draw']]
away_df2['Win_Points'] = np.where(df['away_win'] > 0, 3, 0)
away_df2['Draw_Points'] = np.where(df['draw'] > 0, 1, 0)
away_df2['Match_Points'] = away_df2['Win_Points'] + away_df2['Draw_Points']
away_df3 = away_df2[['match_Id','date','away_team_Id','Match_Points']]
away_df3.rename(columns={'away_team_Id':'team_Id'}, inplace = True)





c:\users\faisal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\faisal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\faisal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [5]:



#functions which output a list representing the 'form' points for last 3 & 6 games

def Get_Last_3_Form(Some_df):
    Match_Points_Column = Some_df['Match_Points']
    Points_List = list(Match_Points_Column)
    form_3 = [sum(Points_List[max(0, i - 3):i]) 
              for i in range(len(Points_List))]
    return form_3

def Get_Last_6_Form(Some_df):
    Match_Points_Column = Some_df['Match_Points']
    Points_List = list(Match_Points_Column)
    form_6 = [sum(Points_List[max(0, i - 6):i]) 
              for i in range(len(Points_List))]
    return form_6


#splitting dataframe, applying 'form' functions, merging results

frames = [home_df3, away_df3]
points_df = pd.concat(frames)
points_df = points_df.sort_values(by = ['team_Id','date']) 
points_by_team = points_df.groupby('team_Id')


split_points_df = [points_by_team.get_group(x) for x in points_by_team.groups]
split_points_df

Form_column_3 = [Get_Last_3_Form(team) for team in split_points_df ]
Form_column_6 = [Get_Last_6_Form(team) for team in split_points_df ]

# concatinating lists as elements within a list into a single list

Final_Form_3 = [inner for outer in Form_column_3 for inner in outer]
Final_Form_6 = [inner for outer in Form_column_6 for inner in outer]
points_df['Form_3'] = Final_Form_3
points_df['Form_6'] = Final_Form_6



In [6]:
#Joining form dataframe to master dataframe

df_3 = pd.merge(df_2, 
                points_df,
                left_on = ['match_Id', 'home_team_Id'],
                right_on = ['match_Id', 'team_Id'],
                how = 'left'
               )
df_3 = df_3.drop(['date_y','team_Id','Match_Points'], axis = 1)
df_3.rename(columns={'Form_3':'home_Form_3', 'Form_6':'home_Form_6', 'date_x':'date'}, inplace = True)

df_3 = pd.merge(df_3, 
                points_df,
                left_on = ['match_Id', 'away_team_Id'],
                right_on = ['match_Id', 'team_Id'],
                how = 'left'
               )
df_3 = df_3.drop(['date_y','team_Id','Match_Points'], axis = 1)
df_3.rename(columns={'Form_3':'away_Form_3', 'Form_6':'away_Form_6', 'date_x':'date'}, inplace = True)

In [8]:
df_3.head()

,match_Id,date,home_team_Id,home_team,away_team_Id,away_team,home_score,away_score,country,city,tournament,neutral,home_win,away_win,draw,home_Form_3,home_Form_6,away_Form_3,away_Form_6
0,1,1872-11-30,239,Scotland,86,England,0,0,Scotland,Glasgow,Friendly,False,0,0,1,0,0,0,0
1,2,1873-03-08,86,England,239,Scotland,4,2,England,London,Friendly,False,1,0,0,1,1,1,1
2,3,1874-03-07,239,Scotland,86,England,2,1,Scotland,Glasgow,Friendly,False,1,0,0,1,1,4,4
3,4,1875-03-06,86,England,239,Scotland,2,2,England,London,Friendly,False,0,0,1,4,4,4,4
4,5,1876-03-04,239,Scotland,86,England,3,0,Scotland,Glasgow,Friendly,False,1,0,0,4,5,4,5


In [15]:
#Teams_df.head()

Elo_Df = pd.DataFrame(index = df_3['date'], columns = Teams_df['Id'])

Elo_Df.head()

Id,1,2,3,4,5,6,7,8,9,10,...,304,305,306,307,308,309,310,311,312,313
date,,,,,,,,,,,,,,,,,,,,,
1872-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1874-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
